In [15]:
import os
import json
import nibabel as nib
import numpy as np
import torch
import torchvision
import torchio as tio
import matplotlib.pyplot as plt

# global settings
base_path = '/mnt/data_lab513/vqtran_data'
root_data = os.path.join(base_path, "data", "raw_data", "ADNI_NIfTI")
root_bias_correction = os.path.join(base_path, "data", "clean_data", "mri_bias_correction")
root_bet = os.path.join(base_path, "data", "clean_data", "mri_brain_extraction")
root_reg = os.path.join(base_path, "data", "clean_data", "mri_registration")
root_meta = os.path.join(base_path, "data", "meta_data")#, "Pre-Thesis_metadata", "ADNI") 
root_train = os.path.join(base_path, "data", "train_data")
root_train_dec = os.path.join(base_path, "data", "data_train_dec", "origin")
root_train_unique = os.path.join(base_path, "data", "data_train_dec", "unique")
root_train_unique_tensor = os.path.join(base_path, "data", "data_train_dec", "tensor")

RESIZE AND SAVE TORCH DATASETS

In [16]:

# subject_dict = json.load(open('investigate/unique_dataset_dict.json', 'r'))
subject_dict = json.load(open('unique_dataset_dict.json', 'r'))

In [17]:
# print(subject_dict)
X_tensor_cross_val = []
Y_tensor_cross_val = []

for key in subject_dict.keys():
    filename = subject_dict[key][2]
    image_absolute_path = os.path.join(root_train_unique, filename)
    # print(filename)
    label = subject_dict[key][1]
    # print(label)
    
    np_label = -1
    if label == "CN" or label == "AD":
        if label == "CN":
            np_label = np.array([0])
        elif label == "AD":
            np_label = np.array([1])
        else:
            raise ValueError("label must be CN or AD")  
    else:
        continue
        # raise ValueError("label can not be EMCI or LMCI")
    
    image_sample = nib.load(image_absolute_path)
    image_array = image_sample.get_fdata()
    
    image_tensor = torch.Tensor(image_array)
    image_tensor = torch.unsqueeze(image_tensor,0)
    
    image_transformation_tio = tio.transforms.Compose(
            [
                tio.transforms.Resize((110,110,110)),
                tio.ZNormalization(),
                tio.RescaleIntensity(out_min_max=(0, 1)) #), in_min_max=(0., 8957.8574))
            ]
    )

    image_tensor = image_transformation_tio(image_tensor)

    # print(image_tensor.shape)
    

    
    label_tensor = torch.Tensor(np_label)

    X_tensor_cross_val.append(image_tensor)
    Y_tensor_cross_val.append(label_tensor)
    
X_tensor_cross_val = torch.stack(X_tensor_cross_val)
Y_tensor_cross_val = torch.stack(Y_tensor_cross_val)

Y_tensor_cross_val = Y_tensor_cross_val.ravel()
print(X_tensor_cross_val.shape)
print(Y_tensor_cross_val.shape)


torch.Size([627, 1, 110, 110, 110])
torch.Size([627])


In [18]:
torch.save(X_tensor_cross_val, os.path.join(root_train_unique_tensor, "x_tensor_NC_AD_cv.pt"))
torch.save(Y_tensor_cross_val, os.path.join(root_train_unique_tensor, "y_tensor_NC_AD_cv.pt"))

CALCULATE MEAN AND STD OF IMAGES

In [19]:
tensor_mean = torch.mean(X_tensor_cross_val) #tensor(86.5368) #tensor(0.1287)
tensor_std = torch.std(X_tensor_cross_val) #tensor(258.8274) #tensor(0.2440)
tensor_max = torch.max(X_tensor_cross_val) #tensor(8957.8574) #tensor(1.)
tensor_min = torch.min(X_tensor_cross_val) #tensor(0.)  #tensor(0.)
tensor_unique, tensor_count = torch.unique(Y_tensor_cross_val, return_counts=True)

# tensor_unique # tensor([0., 1.])
# tensor_count # tensor([349, 278])

In [20]:
print(tensor_mean)
print(tensor_std)
print(tensor_max)
print(tensor_min)
print(tensor_unique)
print(tensor_count)

tensor(0.1287)
tensor(0.2440)
tensor(1.)
tensor(0.)
tensor([0., 1.])
tensor([349, 278])


In [21]:
print(root_train_unique_tensor)

/mnt/data_lab513/vqtran_data/data/data_train_dec/tensor


In [22]:
import torch
y_tensor = torch.load(os.path.join("/mnt/data_lab513/vqtran_data/data_torch_tensor/tensor","y_tensor_NC_AD_cv.pt"))

In [23]:
print(y_tensor.shape)

torch.Size([628])


In [24]:
print(y_tensor[:10])

tensor([0, 0, 0, 0, 0, 1, 0, 0, 1, 1])


In [25]:
y_tensor_new = torch.load(os.path.join(root_train_unique_tensor, "y_tensor_NC_AD_cv.pt"))

In [27]:
print(y_tensor_new[:10])

tensor([0., 0., 0., 1., 0., 1., 1., 1., 0., 0.])
